# Interactivity

In [9]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from ipywidgets import widgets, Layout
import folium
from folium.plugins import MarkerCluster
from IPython.display import clear_output


# Get and Collapse Data by State, County

In [12]:
# Get cleaned gun violence master dataset and states data
directory = os.path.dirname(str(Path().resolve()) + '\\')
path = os.path.join(directory, 'data', 'final_data.csv')
gunViolence = pd.read_csv(path)

path = os.path.join(directory, 'data', 'county_unemployment.csv')
countyData = pd.read_csv(path)

In [14]:
countyData.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [5]:
stateCols = ['state', 'Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 
             'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score',  'Gun Deaths per 100k People (2018)', 
             'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type',  
             'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score']

# countyCols = ['']

stateData = gunViolence[stateCols].drop_duplicates('state')



# Display Interactive Map with Data by State (Most Recent Code)

In [6]:
# embed_map function written by user: ocefpaf to patch folium rendering issue with > 80 objects in Google Chrome
def embed_map(m):
    from IPython.display import IFrame
    m.save('index.html')
    return IFrame('index.html', width='100%', height='650px')

# Define list of factors that should be plotted with high values as red instead of green
badFactorsList = ['Gun_Death_Rate_2018 (per 100k people)', '# of guns per capita', '# of guns registered',]
def getColColor(col):
    if (col in badFactorsList):
        return 'OrRd'
    else:
        return 'YlGn'
    
def getMarkerColor(n_killed):
    if (n_killed == 0):
        return '#ffff00'
    elif (n_killed < 2):
        return '#f8d568'
    elif (n_killed < 5):
        return '#ffa500'
    else:
        return '#ff0000'

In [7]:
dropdown = widgets.Dropdown(
    options=stateData.columns.values[1:],
    value='5_Year_Cost',
    description='Factor to Plot:',
    disabled=False,
    layout=Layout(width='40%'),
    style={'description_width': 'initial'}
)

slider = widgets.IntSlider(min=0,max=1000,step=100,value=500,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
button = widgets.Button(description = "Go!")
display(dropdown)
display(slider)
display(button)


def on_button_clicked(b):
    clear_output()
    display(dropdown)
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97], zoom_start=4, tiles="Mapbox Bright")   
    cluster = MarkerCluster(name='Gun Violence Incidents')
    for i, row in gunViolence.sample(frac=1).head(slider.value).iterrows():
        cluster.add_child(folium.CircleMarker(location=[row.latitude,row.longitude], radius =5*(row.n_killed+row.n_injured), fill=True, color=getMarkerColor(row.n_killed), popup=folium.Popup('Date: '+str(row.month)+'-'+str(row.day)+'-'+str(row.year)+'\nLives Lost: '+str(row.n_killed),max_width=200)))
    folium_map.add_child(cluster)
    
    if (dropdown.value == 'Unemployment'):
        folium.Choropleth(
            geo_data='data/us_counties_20m.json',
            name=dropdown.value,
            data=countyData,
            columns=['state', dropdown.value],
            key_on='feature.properties.NAME',
            fill_color=getColColor(dropdown.value),
            fill_opacity=0.5,
            line_opacity=0.2,
            legend_name=dropdown.value
        ).add_to(folium_map)
    else:
        folium.Choropleth(
            geo_data='data/us_states_20m.json',
            name=dropdown.value,
            data=stateData,
            columns=['state', dropdown.value],
            key_on='feature.properties.NAME',
            fill_color=getColColor(dropdown.value),
            fill_opacity=0.5,
            line_opacity=0.2,
            legend_name=dropdown.value
        ).add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    toDisp = embed_map(folium_map)
    display(toDisp)

button.on_click(on_button_clicked)

Dropdown(description='Factor to Plot:', index=12, layout=Layout(width='40%'), options=('Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score', 'Gun Deaths per 100k People (2018)', 'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type', 'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score'), style=DescriptionStyle(description_width='initial'), value='5_Year_Cost')

IntSlider(value=100, description='Number of Incidents:', layout=Layout(width='40%'), max=1000, step=100, style=SliderStyle(description_width='initial'))

Button(description='Go!', style=ButtonStyle())

# Subcomponent Demonstration Code

In [8]:
# Testing for Marker Clusters
# Create interactive widgets with mapping function
dropdown = widgets.Dropdown(
    options=stateData.columns.values[1:],
    value='5_Year_Cost',
    description='Factor to Plot:',
    disabled=False,
    layout=Layout(width='40%'),
    style={'description_width': 'initial'}
)

slider = widgets.IntSlider(min=0,max=1000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
button = widgets.Button(description = "Go!")
display(dropdown)
display(slider)
display(button)


def on_button_clicked(b):
    clear_output()
    display(dropdown)
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97], zoom_start=4, tiles="Mapbox Bright")   
    cluster = MarkerCluster()
    for i, row in gunViolence.sample(frac=1).head(slider.value).iterrows():
        cluster.add_child(folium.CircleMarker(location=[row.latitude,  row.longitude], radius = 5*(row.n_killed + row.n_injured), popup = 'Lives Lost: ' + str(row.n_killed)))
    folium_map.add_child(cluster)
    folium.Choropleth(
        geo_data='data/us_states_20m.json',
        name=dropdown.value,
        data=stateData,
        columns=['state', dropdown.value],
        key_on='feature.properties.NAME',
        fill_color=getColor(dropdown.value),
        fill_opacity=0.5,
        line_opacity=0.2,
        legend_name=dropdown.value
    ).add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    toDisp = embed_map(folium_map)
    display(toDisp)

button.on_click(on_button_clicked)

Dropdown(description='Factor to Plot:', index=12, layout=Layout(width='40%'), options=('Mental_Health_Records_Submitted_2008', 'Mental_Health_Records_Submitted_2017', 'Gun_Sale_Denials_2008', 'Gun_Sale_Denials_2017', 'Giffords Gun Safety Score', 'Gun Deaths per 100k People (2018)', 'Gun Death Rate (Ranked High to Low)', '# of guns per capita', '# of guns registered', 'Permit Type', 'Handgun_Carry_Permit_Fee', 'Years_Valid', '5_Year_Cost', 'Happiness Score'), style=DescriptionStyle(description_width='initial'), value='5_Year_Cost')

IntSlider(value=100, description='Number of Incidents:', layout=Layout(width='40%'), max=1000, step=100, style=SliderStyle(description_width='initial'))

Button(description='Go!', style=ButtonStyle())

In [12]:
# Simple Overlay Demonstration (no Data)

def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
folium.GeoJson('data/us_counties_20m.json', name='US Counties').add_to(folium_map)
folium.GeoJson('data/us_states_20m.json', name='US States').add_to(folium_map)
folium.LayerControl().add_to(folium_map)
embed_map(folium_map)

In [13]:
# Widget Demonstration Code

button = widgets.Button(description = "Go!")
slider = widgets.IntSlider(min=0,max=5000,step=100,value=100,description='Number of Incidents:',layout=Layout(width='40%'),style={'description_width': 'initial'})
display(slider)
display(button)

def on_button_clicked(b):
    clear_output()
    display(slider)
    display(button)
    folium_map = folium.Map(location=[35, -97],
                            zoom_start=4,
                            tiles="Mapbox Bright")
    for i, row in gunViolence.head(slider.value).iterrows():
        marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], radius = (row['n_killed'] + row['n_injured']))
        marker.add_to(folium_map)
    embed_map(folium_map)
button.on_click(on_button_clicked)

IntSlider(value=2400, description='Number of Incidents:', layout=Layout(width='40%'), max=5000, step=100, styl…

Button(description='Go!', style=ButtonStyle())

In [20]:
folium_map = folium.Map(location=[35, -97],
                        zoom_start=4,
                        tiles="Mapbox Bright")
for i, row in gunViolence.sample(frac=1).head(10000).iterrows():
    marker = folium.CircleMarker(location=[row.latitude,row.longitude], radius =2*(row.n_killed+row.n_injured), fill=True, color=getMarkerColor(row.n_killed), popup=folium.Popup('Date: '+str(row.month)+'-'+str(row.day)+'-'+str(row.year)+'\nLives Lost: '+str(row.n_killed),max_width=200))
    marker.add_to(folium_map)
embed_map(folium_map)